In [1]:
include("../LiPoSID.jl")
using QuantumOptics
basis = NLevelBasis(2)
using LinearAlgebra
using DynamicPolynomials
using Dates
using TSSOS
using HDF5
using Formatting

fmt12 = generate_formatter("%1.12f")

function solution_from_operator_sum(E)

    sol = []

    for e in E
        # push α
        push!(sol, real(e[1,1]))
        push!(sol, real(e[2,1]))
        push!(sol, real(e[1,2]))
        push!(sol, real(e[2,2]))
    end

    for e in E
        # push β 
        push!(sol, imag(e[1,1]))
        push!(sol, imag(e[2,1]))
        push!(sol, imag(e[1,2]))
        push!(sol, imag(e[2,2]))
    end

    return convert.(Float64, sol)

end

import Base.real

function rm_micro_coefs(p::AbstractPolynomial, cutoff = 1e-7)
    max_coef = maximum(abs.(coefficients(p)))
    
    sum(
        [ abs(coef)/max_coef > cutoff ? real(coef) * mon : 0 for (coef, mon) in zip(coefficients(p), monomials(p))]
        )
end

function scale_poly(p::Polynomial)
    X = transpose(hcat([exponents(t) for t in terms(p)]...))

    # Get the scaling via linear regression
    scaling = X \ log.(abs.(coefficients(p)))

    scale = exp.(abs.(scaling))

    # scale the polynomial
    p_scaled = subs(p, variables(p) => scale .* variables(p))

    return p_scaled
end



scale_poly (generic function with 1 method)

For all γ: load spin-boson data, simulate LME data, perform 4d DMD, QPT, POP and chech fidelity for 20 states  

In [2]:
parentdir = pwd()
#data_dir = parentdir*"../DATA/"
data_dir = "../DATA/"
println(data_dir)

models_dir = "../MODELS/"
tests_dir = "../TESTS/"

#models_dir = parentdir*"../MODELS/"
#tests_dir = parentdir*"../TESTS/"

dodeca_files = ["State_D"*string(n) for n=1:20];
basis_files = ["State_B"*string(n) for n=1:4];

test_files = dodeca_files;

γ_list = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133",  "79.477", "251.33"]

date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))
models_file = "POP-Kraus_vs_DMD_trn4_obj_models_"*date_and_time_string * ".h5"
tests_data_file_name = "POP-Kraus_vs_DMD_trn4-tst20_obj_fids_"*date_and_time_string * ".h5"

println(models_file)

for γᵢ in γ_list

    println("γ = ", γᵢ)

    if γᵢ == "0.0" 
        ρᵉ, tᵉ = LiPoSID.get_rho_series(data_dir*"State_B2_2CUT_data.h5", "0.079477")
        ρᵍ, tᵍ = LiPoSID.get_rho_series(data_dir*"State_B1_2CUT_data.h5", "0.079477")
        ρˣ, tˣ = LiPoSID.get_rho_series(data_dir*"State_B3_2CUT_data.h5", "0.079477")
        ρʸ, tʸ = LiPoSID.get_rho_series(data_dir*"State_B4_2CUT_data.h5", "0.079477")
    else 
        ρᵉ, tᵉ = LiPoSID.get_rho_series(data_dir*"State_B2_2CUT_data.h5", γᵢ)
        ρᵍ, tᵍ = LiPoSID.get_rho_series(data_dir*"State_B1_2CUT_data.h5", γᵢ)
        ρˣ, tˣ = LiPoSID.get_rho_series(data_dir*"State_B3_2CUT_data.h5", γᵢ)
        ρʸ, tʸ = LiPoSID.get_rho_series(data_dir*"State_B4_2CUT_data.h5", γᵢ)
    end

    lᵉ = length(ρᵉ); lᵍ = length(ρᵍ); lˣ = length(ρˣ); lʸ = length(ρʸ)
    lᵐᵃˣ = min(lᵉ, lᵍ,  lˣ, lʸ)  #choose time limit by shortest series

    ωᶠ = 25.126

    Hᴸᴹᴱ = [ ωᶠ       0
             0        0   ]

    γᶠ = parse(Float64, γᵢ)

    Aᴸᴹᴱ = [ 0    √γᶠ
             0     0. + 0im  ]

    tᵉᶠ = convert.(Float64, tᵉ); tᵍᶠ = convert.(Float64, tᵍ); 
    tˣᶠ = convert.(Float64, tˣ); tʸᶠ = convert.(Float64, tʸ); 

    t = tˣᶠ
    Δt = t[2] - t[1] #0.02

    ρᵉᴸᴹᴱ = LiPoSID.Lindblad_time_evolution(basis, ρᵉ[1], tᵉᶠ, Hᴸᴹᴱ, [Aᴸᴹᴱ])
    ρᵍᴸᴹᴱ = LiPoSID.Lindblad_time_evolution(basis, ρᵍ[1], tᵍᶠ, Hᴸᴹᴱ, [Aᴸᴹᴱ])
    ρˣᴸᴹᴱ = LiPoSID.Lindblad_time_evolution(basis, ρˣ[1], tˣᶠ, Hᴸᴹᴱ, [Aᴸᴹᴱ])
    ρʸᴸᴹᴱ = LiPoSID.Lindblad_time_evolution(basis, ρʸ[1], tʸᶠ, Hᴸᴹᴱ, [Aᴸᴹᴱ])

    Aᴰᴹᴰ⁻ˢᴮ = LiPoSID.direct_DMD_01XY_b4_A([ρᵉ[1:lᵐᵃˣ], ρᵍ[1:lᵐᵃˣ], ρˣ[1:lᵐᵃˣ], ρʸ[1:lᵐᵃˣ]])
    Aᴰᴹᴰ⁻ᴸᴹᴱ = LiPoSID.direct_DMD_01XY_b4_A([ρᵉᴸᴹᴱ[1:lᵐᵃˣ], ρᵍᴸᴹᴱ[1:lᵐᵃˣ], ρˣᴸᴹᴱ[1:lᵐᵃˣ], ρʸᴸᴹᴱ[1:lᵐᵃˣ]])

    ρᵍ₀ = [1. 0]' * [1 0]
    ρᵉ₀ = [0. 1]' * [0 1]
    ρˣ₀ = [1  1]' * [1 1]/2
    ρʸ₀ = [1 -im]' * [1 -im]/2

    ρᵍ₁ᴸᴹᴱ = LiPoSID.DMD_step(Aᴰᴹᴰ⁻ᴸᴹᴱ, ρᵍ₀)
    ρᵉ₁ᴸᴹᴱ = LiPoSID.DMD_step(Aᴰᴹᴰ⁻ᴸᴹᴱ, ρᵉ₀)
    ρˣ₁ᴸᴹᴱ = LiPoSID.DMD_step(Aᴰᴹᴰ⁻ᴸᴹᴱ, ρˣ₀)
    ρʸ₁ᴸᴹᴱ = LiPoSID.DMD_step(Aᴰᴹᴰ⁻ᴸᴹᴱ, ρʸ₀)

    ρᵍ₁ˢᴮ = LiPoSID.DMD_step(Aᴰᴹᴰ⁻ˢᴮ, ρᵍ₀)
    ρᵉ₁ˢᴮ = LiPoSID.DMD_step(Aᴰᴹᴰ⁻ˢᴮ, ρᵉ₀)
    ρˣ₁ˢᴮ = LiPoSID.DMD_step(Aᴰᴹᴰ⁻ˢᴮ, ρˣ₀)
    ρʸ₁ˢᴮ = LiPoSID.DMD_step(Aᴰᴹᴰ⁻ˢᴮ, ρʸ₀)

    Eᴰᴹᴰ⁻ᴸᴹᴱ, dᴸᴹᴱ = LiPoSID.QPT(ρᵍ₁ᴸᴹᴱ, ρᵉ₁ᴸᴹᴱ, ρˣ₁ᴸᴹᴱ, ρʸ₁ᴸᴹᴱ)
    Eᴰᴹᴰ⁻ˢᴮ, dˢᴮ = LiPoSID.QPT(ρᵍ₁ˢᴮ, ρᵉ₁ˢᴮ, ρˣ₁ˢᴮ, ρʸ₁ˢᴮ)

    operators_num = length(Eᴰᴹᴰ⁻ˢᴮ) # 3

    @polyvar α[1:2, 1:2, 1:operators_num]
    @polyvar β[1:2, 1:2, 1:operators_num]

    Kₛ = []

    for i in 1:operators_num
        K = α[:,:,i] + im * β[:,:,i]
        push!(Kₛ, K)
    end

    objᵍ = LiPoSID.kraus_obj(ρᵍ, Kₛ)
    objᵉ = LiPoSID.kraus_obj(ρᵉ, Kₛ)
    objˣ = LiPoSID.kraus_obj(ρˣ, Kₛ)
    objʸ = LiPoSID.kraus_obj(ρʸ, Kₛ)

    objˢᴮ =  objᵍ + objᵉ + objˣ + objʸ
    objˢᴮₛ = objˢᴮ
    #objˢᴮₛ = rm_micro_coefs(objˢᴮ)

    objᵍᴸᴹᴱ = LiPoSID.kraus_obj(ρᵍᴸᴹᴱ, Kₛ)
    objᵉᴸᴹᴱ = LiPoSID.kraus_obj(ρᵉᴸᴹᴱ, Kₛ)
    objˣᴸᴹᴱ = LiPoSID.kraus_obj(ρˣᴸᴹᴱ, Kₛ)
    objʸᴸᴹᴱ = LiPoSID.kraus_obj(ρʸᴸᴹᴱ, Kₛ)

    objᴸᴹᴱ =  objᵍᴸᴹᴱ + objᵉᴸᴹᴱ + objˣᴸᴹᴱ + objʸᴸᴹᴱ
    objᴸᴹᴱₛ = objᴸᴹᴱ
    #objᴸᴹᴱₛ = rm_micro_coefs(objᴸᴹᴱ)

    constr = LiPoSID.frobenius_norm2(sum(k' * k for k in Kₛ) - I)

    solᴰᴹᴰ⁻ˢᴮ = solution_from_operator_sum(Eᴰᴹᴰ⁻ˢᴮ)
    solᴰᴹᴰ⁻ᴸᴹᴱ = solution_from_operator_sum(Eᴰᴹᴰ⁻ᴸᴹᴱ)

    println("------ Spin-boson:")
    
    opt,solᴾᴼᴾ⁻ˢᴮ,dataᴾᴼᴾ⁻ˢᴮ = tssos_first([objˢᴮₛ, constr], variables(objˢᴮ), maxdegree(objˢᴮ)÷2, numeq=1, solution=true, QUIET=true)
    sb_tssos_higher = false
    previous_solᴾᴼᴾ⁻ˢᴮ = solᴾᴼᴾ⁻ˢᴮ
    SDP_status1ᴾᴼᴾ⁻ˢᴮ = dataᴾᴼᴾ⁻ˢᴮ.SDP_status

    while ~isnothing(solᴾᴼᴾ⁻ˢᴮ)
        println("--- TSSOS higher output: ---")
        sb_tssos_higher = true
        previous_solᴾᴼᴾ⁻ˢᴮ = solᴾᴼᴾ⁻ˢᴮ
        opt,solᴾᴼᴾ⁻ˢᴮ,dataᴾᴼᴾ⁻ˢᴮ = tssos_higher!(dataᴾᴼᴾ⁻ˢᴮ; QUIET=true, solution=true);
        SDP_status2ᴾᴼᴾ⁻ˢᴮ = dataᴾᴼᴾ⁻ˢᴮ.SDP_status
        println("--- TSSOS higher output end ---")
    end
    
    solutionᴰᴹᴰ⁻ˢᴮ = variables(objˢᴮ) => solᴰᴹᴰ⁻ˢᴮ
    solutionᴾᴼᴾ⁻ˢᴮ = variables(objˢᴮ) => previous_solᴾᴼᴾ⁻ˢᴮ


    println("------ LME:")
    
    opt,solᴾᴼᴾ⁻ᴸᴹᴱ,dataᴾᴼᴾ⁻ᴸᴹᴱ = tssos_first([objᴸᴹᴱₛ, constr], variables(objᴸᴹᴱ), maxdegree(objᴸᴹᴱ)÷2, numeq=1, solution=true, QUIET=true)
    lme_tssos_higher = false
    previous_solᴾᴼᴾ⁻ᴸᴹᴱ = solᴾᴼᴾ⁻ᴸᴹᴱ
    SDP_status1ᴾᴼᴾ⁻ᴸᴹᴱ = dataᴾᴼᴾ⁻ᴸᴹᴱ.SDP_status
    while ~isnothing(solᴾᴼᴾ⁻ᴸᴹᴱ)
        println("--- TSSOS higher output: ---")
        lme_tssos_higher = true
        previous_solᴾᴼᴾ⁻ᴸᴹᴱ = solᴾᴼᴾ⁻ᴸᴹᴱ
        opt,solᴾᴼᴾ⁻ᴸᴹᴱ,dataᴾᴼᴾ⁻ᴸᴹᴱ = tssos_higher!(dataᴾᴼᴾ⁻ᴸᴹᴱ; QUIET=true, solution=true);
        SDP_status2ᴾᴼᴾ⁻ᴸᴹᴱ = dataᴾᴼᴾ⁻ᴸᴹᴱ.SDP_status
        println("--- TSSOS higher output end ---")
    end
    
    
    solutionᴰᴹᴰ⁻ᴸᴹᴱ = variables(objᴸᴹᴱ) => solᴰᴹᴰ⁻ᴸᴹᴱ
    solutionᴾᴼᴾ⁻ᴸᴹᴱ = variables(objᴸᴹᴱ) => previous_solᴾᴼᴾ⁻ᴸᴹᴱ

    Eᴾᴼᴾ⁻ˢᴮ = [convert.(ComplexF64, subs(k, solutionᴾᴼᴾ⁻ˢᴮ)) for k in Kₛ]
    Eᴾᴼᴾ⁻ᴸᴹᴱ = [convert.(ComplexF64, subs(k, solutionᴾᴼᴾ⁻ᴸᴹᴱ)) for k in Kₛ]

    obj_val_POP_sb = convert(Float64, subs(objˢᴮ, solutionᴾᴼᴾ⁻ˢᴮ))
    obj_val_DMD_sb = convert(Float64, subs(objˢᴮ, solutionᴰᴹᴰ⁻ˢᴮ))
    obj_val_POP_LME = convert(Float64, subs(objᴸᴹᴱ, solutionᴾᴼᴾ⁻ᴸᴹᴱ))
    obj_val_DMD_LME = convert(Float64, subs(objᴸᴹᴱ, solutionᴰᴹᴰ⁻ᴸᴹᴱ))


    h5open(models_dir*models_file,"cw") do fid  # read-write, create file if not existing, preserve existing contents

        γ_group = create_group(fid, "gamma_"*γᵢ)   
        
        for i in 1:operators_num

            γ_group["E_DMD-LME_"*string(i)] = [convert.(ComplexF64, e) for e in Eᴰᴹᴰ⁻ᴸᴹᴱ][i]
            γ_group["E_DMD-sb_"*string(i)] =  [convert.(ComplexF64, e) for e in Eᴰᴹᴰ⁻ˢᴮ][i]
            γ_group["E_POP-LME_"*string(i)] = Eᴾᴼᴾ⁻ᴸᴹᴱ[i]
            γ_group["E_POP-sb_"*string(i)] = Eᴾᴼᴾ⁻ˢᴮ[i]

        end

        γ_group["SDP_status1_pop-sb"] = string("SDP_status1ᴾᴼᴾ⁻ˢᴮ")
        γ_group["SDP_status2_pop-sb"] = string("SDP_status2ᴾᴼᴾ⁻ˢᴮ")
        γ_group["SDP_status1_pop-lme"] = string("SDP_status1ᴾᴼᴾ⁻ᴸᴹᴱ")
        γ_group["SDP_status2_pop-lme"] = string("SDP_status2ᴾᴼᴾ⁻ᴸᴹᴱ")
        
        γ_group["d_QPT_LME"] = dᴸᴹᴱ
        γ_group["d_QPT_sb"] = dˢᴮ

        γ_group["sb_tssos_higher"] = sb_tssos_higher
        γ_group["lme_tssos_higher"] = lme_tssos_higher

        γ_group["obj_val_DMD_LME"] = obj_val_DMD_LME
        γ_group["obj_val_DMD_sb"] =  obj_val_DMD_sb
        γ_group["obj_val_POP_LME"] = obj_val_POP_LME
        γ_group["obj_val_POP_sb"] = obj_val_POP_sb
        
        #γ_group["A_sid_lme"] = convert.(ComplexF64, Aˢⁱᵈₗₘₑ)
        
    end # of HDF5 writing

    h5open(tests_dir*tests_data_file_name,"cw") do fid
        γ_group = create_group(fid, "gamma_"*γᵢ)
    end 

    println()
    println(" Minimum fidelities vs spin-boson of ")
    println("         POP:      DMD:     Kraus-corrected DMD:" )

    start_time = time()

    for df in test_files # loop over initial states        

        ρₛ, tₛ = LiPoSID.get_rho_series(data_dir*df*"_2CUT_data.h5", γᵢ)
        ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)
        ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]
        tᵗˢᵗ = convert(Vector{Float64}, tₛ)
        tₛₜₑₚₛ = min(length(ρᵗˢᵗ), lᵐᵃˣ)
        ρ₀ = ρᵗˢᵗ[1]

        tᴸᴹᴱ, ρᴸᴹᴱ  = timeevolution.master(tᵗˢᵗ, ρ₀, DenseOperator(basis,Hᴸᴹᴱ), [DenseOperator(basis, Aᴸᴹᴱ)])
       
        ρᴰᴹᴰ⁻ᴸᴹᴱ = LiPoSID.propagate_DMD_b4(Aᴰᴹᴰ⁻ᴸᴹᴱ, ρₛ[1], tₛₜₑₚₛ)
        ρᴰᴹᴰ⁻ˢᴮ = LiPoSID.propagate_DMD_b4(Aᴰᴹᴰ⁻ˢᴮ, ρₛ[1], tₛₜₑₚₛ)
        ρᴰᴹᴰ⁻ᴸᴹᴱ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρᴰᴹᴰ⁻ᴸᴹᴱ]
        ρᴰᴹᴰ⁻ˢᴮ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρᴰᴹᴰ⁻ˢᴮ]

        ρᴰᴹᴰ⁻ᴸᴹᴱ⁻ᴷʳᵃᵘˢ = LiPoSID.timeevolution_kraus(tₛₜₑₚₛ, ρₛ[1], Eᴰᴹᴰ⁻ᴸᴹᴱ)
        ρᴰᴹᴰ⁻ˢᴮ⁻ᴷʳᵃᵘˢ = LiPoSID.timeevolution_kraus(tₛₜₑₚₛ, ρₛ[1], Eᴰᴹᴰ⁻ˢᴮ)
        ρᴰᴹᴰ⁻ᴸᴹᴱ⁻ᴷʳᵃᵘˢ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρᴰᴹᴰ⁻ᴸᴹᴱ⁻ᴷʳᵃᵘˢ]
        ρᴰᴹᴰ⁻ˢᴮ⁻ᴷʳᵃᵘˢ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρᴰᴹᴰ⁻ˢᴮ⁻ᴷʳᵃᵘˢ]

        ρᴾᴼᴾ⁻ᴸᴹᴱ⁻ᴷʳᵃᵘˢ = LiPoSID.timeevolution_kraus(tₛₜₑₚₛ, ρₛ[1], Eᴾᴼᴾ⁻ᴸᴹᴱ)
        ρᴾᴼᴾ⁻ˢᴮ⁻ᴷʳᵃᵘˢ = LiPoSID.timeevolution_kraus(tₛₜₑₚₛ, ρₛ[1], Eᴾᴼᴾ⁻ˢᴮ)
        ρᴾᴼᴾ⁻ᴸᴹᴱ⁻ᴷʳᵃᵘˢ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρᴾᴼᴾ⁻ᴸᴹᴱ⁻ᴷʳᵃᵘˢ]
        ρᴾᴼᴾ⁻ˢᴮ⁻ᴷʳᵃᵘˢ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρᴾᴼᴾ⁻ˢᴮ⁻ᴷʳᵃᵘˢ]

        Fᴾᴼᴾ⁻ᵛˢ⁻ˢᴮ  = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᵗˢᵗ, ρᴾᴼᴾ⁻ˢᴮ⁻ᴷʳᵃᵘˢ)]
        Fᴾᴼᴾ⁻ᵛˢ⁻ᴸᴹᴱ = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᴸᴹᴱ, ρᴾᴼᴾ⁻ᴸᴹᴱ⁻ᴷʳᵃᵘˢ)]
        Fᴰᴹᴰ⁻ᵛˢ⁻ˢᴮ  = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᵗˢᵗ, ρᴰᴹᴰ⁻ˢᴮ)]
        Fᴰᴹᴰ⁻ᵛˢ⁻ᴸᴹᴱ = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᴸᴹᴱ, ρᴰᴹᴰ⁻ᴸᴹᴱ)]
        Fᴰᴹᴰ⁻ᴷʳᵃᵘˢ⁻ᵛˢ⁻ˢᴮ  = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᵗˢᵗ, ρᴰᴹᴰ⁻ˢᴮ⁻ᴷʳᵃᵘˢ)]
        Fᴰᴹᴰ⁻ᴷʳᵃᵘˢ⁻ᵛˢ⁻ᴸᴹᴱ = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᴸᴹᴱ, ρᴰᴹᴰ⁻ᴸᴹᴱ⁻ᴷʳᵃᵘˢ)]

        Fᴸᴹᴱₑₓ = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᵗˢᵗ, ρᴸᴹᴱ)]
                    

        h5open(tests_dir*tests_data_file_name,"cw") do fid
            γ_group = open_group(fid, "gamma_"*γᵢ) # open coupling group

            init_state_group = create_group(γ_group, df) # create initial state group

            init_state_group["F_POP-vs-sb"] = convert.(Float64, Fᴾᴼᴾ⁻ᵛˢ⁻ˢᴮ)
            init_state_group["F_POP-vs-LME"] = convert.(Float64, Fᴾᴼᴾ⁻ᵛˢ⁻ᴸᴹᴱ)
            init_state_group["F_DMD-vs-sb"] = convert.(Float64, Fᴰᴹᴰ⁻ᵛˢ⁻ˢᴮ) 
            init_state_group["F_DMD-vs-LME"] = convert.(Float64, Fᴰᴹᴰ⁻ᵛˢ⁻ᴸᴹᴱ) 
            init_state_group["F_DMD-Kraus-vs-sb"] = convert.(Float64, Fᴰᴹᴰ⁻ᴷʳᵃᵘˢ⁻ᵛˢ⁻ˢᴮ) 
            init_state_group["F_DMD-Kraus-vs-LME"] = convert.(Float64, Fᴰᴹᴰ⁻ᴷʳᵃᵘˢ⁻ᵛˢ⁻ᴸᴹᴱ) 

            init_state_group["time"] = convert.(Float64, tᵗˢᵗ[1:tₛₜₑₚₛ])
            
        end
            
        println(df*" ", fmt12(minimum(Fᴾᴼᴾ⁻ᵛˢ⁻ˢᴮ))," ", 
                        fmt12(minimum(Fᴰᴹᴰ⁻ᵛˢ⁻ˢᴮ))," ", 
                        fmt12(minimum(Fᴰᴹᴰ⁻ᴷʳᵃᵘˢ⁻ᵛˢ⁻ˢᴮ)))
            
    end

    print("Test for γ = ", γᵢ)
    
    println(" done in ", time() - start_time )

end

println("All calculations finished.")

println("Tests saved in ", tests_data_file_name)


../DATA/


POP-Kraus_vs_DMD_trn4_obj_models_2023-Dec-20_at_13-56.h5


γ = 0.079477
------ Spin-boson:


*********************************** TSSOS ***********************************


Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is launching...
optimum = 0.00011596498709466529

...

--- TSSOS higher output end ---
--- TSSOS higher output: ---
--- TSSOS higher output end ---

 Minimum fidelities vs spin-boson of 
         POP:      DMD:     Kraus-corrected DMD:
State_D1 

0.998053957682 0.998122861807 0.982958295980
State_D2 0.997202078537 0.997078715166 0.994104084855
State_D3 0.997969608076 0.997860379441 0.983685601573
State_D4 0.997108479955 0.997216263558 0.994520245825
State_D5 0.996613771122 0.996697138470 0.996695671600


State_D6 0.996230421158 0.996093827108 0.995909279951
State_D7 0.998365174908 0.999711431758 0.931859765809
State_D8 0.998873761662 0.999224267831 0.958960273319
State_D9 0.997018597573 0.996993237985 0.996590723564
State_D10 0.996856396270 0.996867828090 0.997079650604
State_D11 0.997768577268 0.997893104888 0.979195600819


State_D12 0.997379091193 0.997190652218 0.991817934484
State_D13 0.997837315200 0.997713107267 0.980097300172
State_D14 0.997461006784 0.997597929508 0.992371677419
State_D15 0.996171799924 0.996290678457 0.996120489000
State_D16 0.996555561670 0.996400172585 0.996782690047
State_D17 

0.995729157337 0.997965045796 0.923963284559
State_D18 0.999056276522 0.999042829078 0.964690407289
State_D19 0.996882532961 0.996866465852 0.994858772053
State_D20 0.997033573418 0.997040642516 0.995545280140
Test for γ = 251.33 done in 0.41752004623413086
All calculations finished.
Tests saved in POP-Kraus_vs_DMD_trn4-tst20_obj_fids_2023-Dec-20_at_13-56.h5
